<a href="https://colab.research.google.com/github/aneeq-shaffy/DL-labsheets/blob/main/paddy_rice_disease_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download(
    "tntiphan/paddy-rice-disease-classification"
)

print(dataset_path)

Using Colab cache for faster access to the 'paddy-rice-disease-classification' dataset.
/kaggle/input/paddy-rice-disease-classification


### Model Testing

In [26]:
import numpy as np
from PIL import Image
import tensorflow as tf

# Load and preprocess the image
image_path = '/content/test1.jpg'
img = Image.open(image_path).resize(IMG_SIZE)
img_array = np.array(img).astype(np.float32)

# EfficientNet preprocessing (already defined in previous cells for train/val_ds)
# Ensure the image array has the correct shape for a batch of 1 image and then preprocess
img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = tf.keras.applications.efficientnet.preprocess_input(img_array)

In [27]:
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="/kaggle/working/rice_disease_effnet.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
out_details = interpreter.get_output_details()

# Set the tensor
interpreter.set_tensor(input_details[0]['index'], preprocessed_img)

# Run inference
interpreter.invoke()

# Get the output prediction
output_data = interpreter.get_tensor(out_details[0]['index'])

# Get predicted class and confidence
predicted_class_id = np.argmax(output_data[0])
confidence = np.max(output_data[0])

predicted_label = id2label[str(predicted_class_id)]

print(f"Predicted Class: {predicted_label}")
print(f"Confidence: {confidence:.2f}")

Predicted Class: leaf_blast
Confidence: 0.56


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [2]:
import json
import os

id2label_path = os.path.join(dataset_path, 'id2label.json')
with open(id2label_path, 'r') as f:
    id2label = json.load(f)

print("id2label.json content:", id2label)

# Also read label2id.json for completeness, as they usually come in pairs
label2id_path = os.path.join(dataset_path, 'label2id.json')
with open(label2id_path, 'r') as f:
    label2id = json.load(f)

print("label2id.json content:", label2id)

id2label.json content: {'0': 'bacterial_leaf_blight', '1': 'brown_spot', '2': 'healthy', '3': 'leaf_blast'}
label2id.json content: {'bacterial_leaf_blight': 0, 'brown_spot': 1, 'healthy': 2, 'leaf_blast': 3}


In [3]:
import pandas as pd
import os

metadata_path = os.path.join(dataset_path, 'metadata.csv')
metadata_df = pd.read_csv(metadata_path)
display(metadata_df.head())

,path,label,split
0,/kaggle/input/paddy-rice-disease-classificatio...,leaf_blast,train
1,/kaggle/input/paddy-rice-disease-classificatio...,brown_spot,train
2,/kaggle/input/paddy-rice-disease-classificatio...,bacterial_leaf_blight,train
3,/kaggle/input/paddy-rice-disease-classificatio...,bacterial_leaf_blight,train
4,/kaggle/input/paddy-rice-disease-classificatio...,brown_spot,train


In [4]:

NUM_CLASSES = 4
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [5]:
import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [8]:
!pip install kagglehub
import kagglehub

dataset_path = kagglehub.dataset_download(
    "tntiphan/paddy-rice-disease-classification"
)

print(dataset_path)

100%|██████████| 4.05G/4.05G [00:47<00:00, 92.4MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/tntiphan/paddy-rice-disease-classification/versions/7


In [9]:
print(dataset_path)

/root/.cache/kagglehub/datasets/tntiphan/paddy-rice-disease-classification/versions/7


In [10]:
import os

os.listdir("/kaggle/input/paddy-rice-disease-classification")


['label2id.json', 'metadata.csv', 'data', 'id2label.json']

In [11]:
import os

os.listdir("/kaggle/input/paddy-rice-disease-classification/data")


['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast']

In [12]:
NUM_CLASSES = 4


In [13]:
import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

base_model.trainable = False


In [14]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])


In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [16]:
import tensorflow as tf

DATASET_PATH = "/kaggle/input/paddy-rice-disease-classification/data"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# EfficientNet preprocessing
from tensorflow.keras.applications.efficientnet import preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds   = val_ds.map(lambda x, y: (preprocess_input(x), y))

# Optional performance boost
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)


Found 15291 files belonging to 4 classes.
Using 12233 files for training.
Found 15291 files belonging to 4 classes.
Using 3058 files for validation.


In [17]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 201s 448ms/step - accuracy: 0.6415 - loss: 0.9078 - val_accuracy: 0.8427 - val_loss: 0.4555
Epoch 2/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 133s 347ms/step - accuracy: 0.8424 - loss: 0.4315 - val_accuracy: 0.8784 - val_loss: 0.3537
Epoch 3/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 121s 292ms/step - accuracy: 0.8607 - loss: 0.3839 - val_accuracy: 0.8898 - val_loss: 0.3189
Epoch 4/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 131s 342ms/step - accuracy: 0.8693 - loss: 0.3549 - val_accuracy: 0.8993 - val_loss: 0.2978
Epoch 5/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 113s 296ms/step - accuracy: 0.8816 - loss: 0.3343 - val_accuracy: 0.8996 - val_loss: 0.2832
Epoch 6/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 132s 345ms/step - accuracy: 0.8824 - loss: 0.3137 - val_accuracy: 0.9009 - val_loss: 0.2807
Epoch 7/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 123s 295ms/step - accuracy: 0.8942 - loss: 0.2980 - val_accuracy: 0.9120 - val_loss: 0.2727
Epoch 8/10
383/383 ━━━━━━━━━━━━━━━━━━━━ 144s 301ms/step - accuracy: 0.8921 -

In [18]:
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc*100:.2f}%")


96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 232ms/step - accuracy: 0.9031 - loss: 0.2847
Validation accuracy: 91.01%


In [19]:
model.save("/kaggle/working/rice_disease_effnet.h5")


In [20]:
import tensorflow as tf

model = tf.keras.models.load_model("/kaggle/working/rice_disease_effnet.h5")


In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("/kaggle/working/rice_disease_effnet.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp0c3dv8d5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  131981407505104: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  131981407508944: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  131981407509904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407507600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407502416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407507792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407510864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407503760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407507024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131981407509520: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [22]:
import os

print(os.path.exists("/kaggle/working/rice_disease_effnet.tflite"))



True


In [23]:
import os

size_mb = os.path.getsize("/kaggle/working/rice_disease_effnet.tflite") / (1024 * 1024)
print(f"{size_mb:.2f} MB")


4.34 MB
